The DBA's are generally concerned about system performance and functioning. One of the important function impacting system performnace is the currently queries running queries on the system. 

Yugabyte provides a feature to track live queries on the system. These queries can be tracked from the Live queries section of Cluster Dashboard, and using the API's.

In this blog, we will focus on capturing the details via API's and analysis of the live queries data. 

We can use this feature to understand following:
1) Currently running queries on the system
2) User Host generating most of the queries
3) Long running queries
4) Total Number of queries

In [59]:
#In following section, we will create a connection to the YB Managed Cloud, which will be further use for executing required API's on DB

#Note: Replace accountId, and token in below section. 

import requests
import json
import pandas as pd
from datetime import datetime, tzinfo
import pytz

accountId="69e28ad7-842c-4001-8cfa-c5c20610f108"

url = "https://cloud.yugabyte.com/api/public/v1/accounts/" + accountId + "/projects"
token=" "

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + token
    }

response = requests.request("GET", url, headers=headers)
#print (response.text)

In [60]:
#Capture the project ID
project_info = json.loads(response.text)
project_id=project_info['data'][0]['id']
#print ("Project ID: " + project_id)

In [61]:
#Capture the Cluster ID
url = "https://cloud.yugabyte.com/api/public/v1/accounts/" + accountId + "/projects/" + project_id + "/clusters"

response = requests.request("GET", url, headers=headers)

cluster_info = json.loads(response.text)
cluster_id=cluster_info['data'][0]['info']['id']

In [62]:
#Capture currently running queries
url = "https://cloud.yugabyte.com/api/public/v1/accounts/" + accountId + "/projects/" + project_id + "/clusters/" + cluster_id + "/live-queries"

response = requests.request("GET", url, headers=headers)

In [63]:
#Capture the live queries in Pandas DataFrame
live_queries = json.loads(response.text)
all_queries=live_queries['data']['ysql']['queries']
df=pd.DataFrame(all_queries)

In [64]:
#Top 5 long running queries based on elapsed_millis
df.sort_values(by='elapsed_millis', ascending=False).head(5)

,id,node_name,db_name,session_status,query,elapsed_millis,query_start_time,app_name,client_host,client_port
1,29052f84-75a4-4323-8046-a33c0536c7cb,kapil-test-single-region-1-n1,northwind,active,insert into pg_class_1 select * from pg_class_1;,23254,2022-09-07 09:23:40.43737+00,ysqlsh,110.235.229.253,59547
5,790cf0af-e488-4956-a3a0-c98424694a8e,kapil-test-single-region-1-n2,yugabyte,active,"INSERT INTO STOCK VALUES ($1, $2, $3, $4, $5, ...",58,2022-09-07 09:24:03.661836+00,PostgreSQL JDBC Driver,110.235.229.253,59597
0,aada0106-6a72-4fa3-b9fc-3dce9b1e427e,kapil-test-single-region-1-n3,yugabyte,idle in transaction,"INSERT INTO STOCK VALUES ($1, $2, $3, $4, $5, ...",0,2022-09-07 09:24:03.522951+00,PostgreSQL JDBC Driver,110.235.229.253,59598
2,458243cd-6729-4001-acce-685d334d4cc8,kapil-test-single-region-1-n1,yugabyte,idle in transaction,"INSERT INTO STOCK VALUES ($1, $2, $3, $4, $5, ...",0,2022-09-07 09:24:03.415383+00,PostgreSQL JDBC Driver,110.235.229.253,59602
3,865b7e49-9fc4-4492-84f2-3e6f35a1fc3e,kapil-test-single-region-1-n1,yugabyte,idle in transaction,"INSERT INTO STOCK VALUES ($1, $2, $3, $4, $5, ...",0,2022-09-07 09:24:03.640606+00,PostgreSQL JDBC Driver,110.235.229.253,59600


In [65]:
# Most number of host generating sessions
df.groupby(by='client_host').size().reset_index(name='counts')

,client_host,counts
0,110.235.229.253,6


In [66]:
# Queries running for more than 10 seconds ( 1 second = 1000 ms)
df [df['elapsed_millis'] >= 10000]

,id,node_name,db_name,session_status,query,elapsed_millis,query_start_time,app_name,client_host,client_port
1,29052f84-75a4-4323-8046-a33c0536c7cb,kapil-test-single-region-1-n1,northwind,active,insert into pg_class_1 select * from pg_class_1;,23254,2022-09-07 09:23:40.43737+00,ysqlsh,110.235.229.253,59547
